In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from IPython import display

import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import pickle
import math
import re

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec, Word2Vec
import logging

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 20
model =  Word2vecMF()
dictionary = {'Evaluation':[],'BFGD':[],'RO':[], 'word2vec':[], 'number of pairs':[]}

In [2]:
model.load_matrices(from_file='enwik-200/matrices2.npz');
sentences=load(from_file='data/x2')
skip = Word2Vec(sentences, size=dimension, min_count=5)

Parsing sentences from training set


In [3]:
Path = 'datasets/men3000.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = correlation(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           index=190000);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = correlation(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           index=360000);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1)
dictionary['RO'].append(score2)
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/men3000.csv
0.20834729057134707 60
0.056389737792431845 60
-0.13107555421771389 60


In [4]:
Path = 'datasets/verb143.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = correlation(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           index=190000);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = correlation(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           index=360000);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1)
dictionary['RO'].append(score2)
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/verb143.csv
0.02277910237891026 11
-0.21412356236175645 11
-0.13211879379767952 11


In [5]:
Path = 'datasets/wordsim353.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = correlation(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           index=190000);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = correlation(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           index=360000);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1)
dictionary['RO'].append(score2)
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/wordsim353.csv
0.25295573380916286 32
-0.17102007220576013 32
0.3193107886199723 32


In [6]:
Path = 'datasets/MTURK-771.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=',');
score1, _, _, chosen_pairs = correlation(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           index=190000);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = correlation(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           index=360000);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1)
dictionary['RO'].append(score2)
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/MTURK-771.csv
0.40262634747915527 30
0.32472738583310534 30
0.46583579064337866 30


In [7]:
Path = 'datasets/mturk287.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = correlation(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           index=190000);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = correlation(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           index=360000);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1)
dictionary['RO'].append(score2)
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/mturk287.csv
0.3696969696969697 10
0.49090909090909085 10
0.7454545454545454 10


In [8]:
Path = 'datasets/rw2034.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = correlation(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           index=190000);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = correlation(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           index=360000);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1)
dictionary['RO'].append(score2)
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/rw2034.csv
1.0 3
0.5 3
0.5 3


In [9]:
Path = 'datasets/simlex999.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = correlation(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           index=190000);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = correlation(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           index=360000);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1)
dictionary['RO'].append(score2)
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/simlex999.csv
-0.04854132917073165 42
-0.2064019455723765 42
-0.08889789332269221 42


In [10]:
df = pd.DataFrame(dictionary)
cols = df.columns.tolist()
j = cols.index('Evaluation')
cols[0],cols[j]=cols[j], cols[0]
j = cols.index('number of pairs')
cols[-1],cols[j]=cols[j], cols[-1]
df=df[cols]
df.style


,Evaluation,BFGD,RO,word2vec,number of pairs
0,men3000,0.208347,0.0563897,-0.131076,60
1,verb143,0.0227791,-0.214124,-0.132119,11
2,wordsim353,0.252956,-0.17102,0.319311,32
3,MTURK-771,0.402626,0.324727,0.465836,30
4,mturk287,0.369697,0.490909,0.745455,10
5,rw2034,1,0.5,0.5,3
6,simlex999,-0.0485413,-0.206402,-0.0888979,42
